In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

/Users/kaylacheng/Desktop/crm intelligence agent/Team2B/.venv/lib/python3.9/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [3]:
from pathlib import Path

# 1) Point to the folder that holds the CSVs
DATA_DIR = Path("data_directory")   # 

# 2) Quick sanity check (should list 5 CSVs)
print("CWD:", Path.cwd())
print("In data_directory:", list(DATA_DIR.glob("*.csv")))

# 3) Read the files
accounts    = pd.read_csv(DATA_DIR / "accounts.csv")
products    = pd.read_csv(DATA_DIR / "products.csv")
pipeline    = pd.read_csv(DATA_DIR / "sales_pipeline.csv")
sales_teams = pd.read_csv(DATA_DIR / "sales_teams.csv")
data_dict   = pd.read_csv(DATA_DIR / "data_dictionary.csv")

for df, name in [(accounts,"Accounts"), (products,"Products"), (pipeline,"Pipeline"), (sales_teams,"Sales Teams")]:
    print(f"\n{name} ---")
    print(df.info())
    print(df.head())

CWD: /Users/kaylacheng/Desktop/crm intelligence agent/Team2B
In data_directory: [PosixPath('data_directory/products.csv'), PosixPath('data_directory/accounts.csv'), PosixPath('data_directory/sales_pipeline.csv'), PosixPath('data_directory/sales_teams.csv'), PosixPath('data_directory/data_dictionary.csv')]

Accounts ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   account           85 non-null     object 
 1   sector            85 non-null     object 
 2   year_established  85 non-null     int64  
 3   revenue           85 non-null     float64
 4   employees         85 non-null     int64  
 5   office_location   85 non-null     object 
 6   subsidiary_of     15 non-null     object 
dtypes: float64(1), int64(2), object(4)
memory usage: 4.8+ KB
None
            account     sector  year_established  revenue  employees  \
0  Acme Corpora

In [5]:
# Ensure date columns are datetime
df['engage_date'] = pd.to_datetime(df['engage_date'], errors='coerce')
df['close_date'] = pd.to_datetime(df['close_date'], errors='coerce')

# Engineered column
df['sales_cycle_days'] = (df['close_date'] - df['engage_date']).dt.days

# Select features
features = [
    'sales_agent', 'manager', 'regional_office',      # team context
    'account', 'sector', 'year_established', 'revenue', 'employees', 'office_location', 'subsidiary_of',  # account context
    'product', 'series', 'sales_price',               # product context
    'deal_stage', 'engage_date', 'close_date', 'close_value', 'sales_cycle_days'  # pipeline context
]
df = df[features]

# Show columns
df.columns
# or
print(df.columns)
# ...existing code...

Index(['sales_agent', 'manager', 'regional_office', 'account', 'sector',
       'year_established', 'revenue', 'employees', 'office_location',
       'subsidiary_of', 'product', 'series', 'sales_price', 'deal_stage',
       'engage_date', 'close_date', 'close_value', 'sales_cycle_days'],
      dtype='object')


In [ ]:
# --- Data Cleaning ---

# 1. Check for missing values
print("Missing values per column:\n", df.isnull().sum())

# 2. Drop rows with too many missing values
df = df.dropna(thresh=int(0.7 * len(df.columns)))  # keeps rows with at least 70% non-NA values

# 3. Fill missing values (example: fill numeric with median, categorical with mode)
for col in df.select_dtypes(include='number'):
    df[col] = df[col].fillna(df[col].median())
for col in df.select_dtypes(include='object'):
    df[col] = df[col].fillna(df[col].mode()[0])

# 4. Remove duplicates
df = df.drop_duplicates()

# 5. Convert date columns to datetime
for date_col in ['engage_date', 'close_date']:
    df[date_col] = pd.to_datetime(df[date_col], errors='coerce')

# 6. Check data types
print("Data types:\n", df.dtypes)

# 7. Quick summary
print(df.describe(include='all'))

Missing values per column:
 sales_agent            0
manager                0
regional_office        0
account             1425
sector              1425
year_established    1425
revenue             1425
employees           1425
office_location     1425
subsidiary_of       7508
product                0
series              1480
sales_price         1480
deal_stage             0
engage_date          500
close_date          2089
close_value         2089
sales_cycle_days    2089
dtype: int64
Data types:
 sales_agent                 object
manager                     object
regional_office             object
account                     object
sector                      object
year_established           float64
revenue                    float64
employees                  float64
office_location             object
subsidiary_of               object
product                     object
series                      object
sales_price                float64
deal_stage                  object
engage

In [7]:
#confirm whether there are still missing values

print("Missing values after cleaning:\n", df.isnull().sum())

Missing values after cleaning:
 sales_agent           0
manager               0
regional_office       0
account               0
sector                0
year_established      0
revenue               0
employees             0
office_location       0
subsidiary_of         0
product               0
series                0
sales_price           0
deal_stage            0
engage_date         133
close_date          633
close_value           0
sales_cycle_days      0
dtype: int64


In [8]:
# Drop rows with missing engage_date or close_date
df = df.dropna(subset=['engage_date', 'close_date'])

print("Missing values after dropping rows with missing dates:\n", df.isnull().sum())

Missing values after dropping rows with missing dates:
 sales_agent         0
manager             0
regional_office     0
account             0
sector              0
year_established    0
revenue             0
employees           0
office_location     0
subsidiary_of       0
product             0
series              0
sales_price         0
deal_stage          0
engage_date         0
close_date          0
close_value         0
sales_cycle_days    0
dtype: int64


In [9]:
df.to_csv("data_directory/cleaned_pipeline.csv", index=False)